In [12]:
import os
import speech_recognition as sr
import csv
from pandas import *

In [3]:
path1 = './videos/'
path2 = './audio/'

# Video file to Audio file

In [ ]:
for filename in os.listdir(path1):
    print(filename)
    if (filename.endswith(".mp4")): #or .avi, .mpeg, whatever.
        id = os.path.splitext(filename)[0]
        os.system("ffmpeg -i {0} {1}.wav".format(os.path.join(path1,filename), os.path.join(path2,id)))
    else:
        continue

# Audio file to Text file

In [9]:
# Run following command for whisper access
# ! pip install git+https://github.com/openai/whisper.git soundfile
r = sr.Recognizer()
header = ['ID', 'Transcription']
f = open('transcriptions.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)

for filename in os.listdir(path2):
    if (filename.endswith(".wav")): #or .avi, .mpeg, whatever.
        audio = sr.AudioFile(os.path.join(path2,filename))
        with audio as source:
            audio = r.record(source)                  
            result = r.recognize_whisper(audio)
            data = [os.path.splitext(filename)[0], result]
            writer.writerow(data)
    else:
        continue

f.close()

# Applying BERT sentiment analysis to extracted text

In [10]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-25 14:06:59.684437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [23]:
# data = read_csv("transcriptions.csv")
# pred_sentences = data['Transcription'].tolist()
f = open('transcriptions.csv', 'r')
file = csv.DictReader(f)
ids = []
pred_sentences = []
for col in file:
    ids.append(col['ID'])
    pred_sentences.append(col['Transcription'])
print (pred_sentences)

[" I'm not gonna tell you who Baby baby baby baby So round you, I love you I'm not gonna tell you who Baby baby I'm shay Cause baby baby So round you, baby I love you, do it, but I'm dead", " There's a new foundation in town, so we're gonna try it. In case you haven't heard, Charlotte Tilbury made Hollywood flawless filter in a foundation form. This is called Charlotte's Beautiful Skin Foundation. Since they're supposedly so similar, we're gonna compare them side by side. Starting off with flawless filter and shade three on the side of my face, we just blended it in, stunning. Now onto Charlotte's beautiful skin. I got shade six, so I hope it matches. Alright blending it in, let's see. You're kidding. Look how stunning. Okay so comparing the two, obviously the foundation has more coverage than the flawless filter, but they both have that really similar radiance, which is Shutskiss. Let's try applying some foundation over the flawless filter. Technically flawless filter is just a primer

In [24]:
import tensorflow as tf

tf_batch = tokenizer(pred_sentences, max_length=512, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
    print(ids[i], ": \n", labels[label[i]])

7152274500090858794 : 
 Negative
7054746897738698031 : 
 Negative
7115762593788071174 : 
 Negative
7086798266838322437 : 
 Negative
7081298765352537350 : 
 Negative
7123195527327698222 : 
 Negative
7099531036937260330 : 
 Negative
7145293556553551150 : 
 Negative
7046066693180919086 : 
 Negative
7017540145020751106 : 
 Negative
